# Código

In [1]:
import os
import sqlite3
import json
import re
from bs4 import BeautifulSoup

diretorio_dados = f'{os.path.abspath("../dados")}{os.path.sep}'
diretorio_legendas = f'{os.path.abspath("../dados/legendas")}{os.path.sep}'
legendas_baixadas = os.listdir(diretorio_legendas)


In [2]:
def carregar_dados_playlist():
    with open(f'{diretorio_dados}playlist.json', 'r') as arquivo:
        return json.load(arquivo)


In [3]:
def carregar_arquivo_ttml(diretorio):
    conteudo = None
    with open(diretorio, 'r') as arquivo:
        conteudo = BeautifulSoup(arquivo.read(), 'html.parser')

    frases = list()
    for paragrafo in conteudo.find_all('p'):
        frases.append([paragrafo.attrs['begin'],
                       paragrafo.attrs['end'],
                       ' '.join(paragrafo.contents)])

    return frases


In [4]:
def carregar_legendas(conexao, legendas, dados):
    resumo = {dado['id']: dado['title']
              for dado in dados['entries'] if dado['title'] != '[Private video]'}
    dados_insercao = list()
    ids_existentes = [id_youtube[0] for id_youtube in conexao.execute(
        'select distinct id_youtube from episodio').fetchall()]

    for legenda in legendas:
        id_video = legenda.split('.')[0]

        if not id_video in ids_existentes:
            titulo = resumo[id_video]
            resultado = re.findall(r'(#\s*)(\d+)', titulo)
            if len(resultado) > 0:
                numero_episodio = re.findall(r'(#\s*)(\d+)', titulo)[0][1]

                for conteudo in carregar_arquivo_ttml(os.path.join(diretorio_legendas, legenda)):
                    conexao.execute('insert into episodio values (?, ?, ?, ?, ?, ?)',
                                    (id_video, numero_episodio, conteudo[0], conteudo[1], titulo, conteudo[2]))

    conexao.commit()


In [5]:
def executar():
    with sqlite3.connect(f'{diretorio_dados}dados_fts.db') as conexao:
        conexao.execute('''
        CREATE VIRTUAL TABLE IF NOT EXISTS episodio USING fts5(
            id_youtube UNINDEXED, 
            numero UNINDEXED, 
            inicio UNINDEXED,
            fim UNINDEXED,
            titulo, 
            conteudo
        );
        ''')

        dados = carregar_dados_playlist()
        carregar_legendas(conexao, legendas_baixadas, dados)
        print('Processo concluído')


In [6]:
executar()

Processo concluído


# Referências

[SQLite FTS5 Extension](https://www.sqlite.org/fts5.html)

[The Spellfix1 Virtual Table](https://www.sqlite.org/spellfix1.html)